# Lamini Classifier Agent Toolkit (LCAT) Example Use
First let's install the Lamini Python SDK.

In [ ]:
pip install lamini --upgrade

Now let's import the Lamini Python SDK and set our API key. You can get your API key from [app.lamini.ai](https://app.lamini.ai/account) - every user gets $300 in free credits to get started.

Checkout our [documentation](https://docs.lamini.ai/authenticate/#1-get-your-lamini-api-key) for other ways to store your api key.

In [ ]:
import lamini

#Uncomment the line below to add api key manually in-line.
#lamini.api_key = "<insert key from app.lamini.ai>"

Now we're all set up. The first thing we do is define a classifier - Lamini uses the concept of a "project" to contain one or many classifier versions for a given task.

When you create a project, you define several key elements that can't be changed later (if we need to make changes, no sweat! creating a new classifier model takes just a few minutes):

1. A project name
2. A base model for the project to use
3. A set of categories or classes or buckets that you like to place items into
4. Descriptions of each category

Projects also (eventually) contain one or many versions of:

1. Example inputs and the expected output - we call this a dataset
2. Models trained on the dataset(s) you've provided

We will use add multiple datasets and create a few models as we iterate and build the best classifier we can!


## Create a Project:

Here we're creating a project named "CustomerSupportTriageExample" that will leverage the Meta-Llama-3.1-8B-Instruct model.

> #### In this notebook we are going to be working through a sample problem of categorizing customer chat logs with an insurance company help desk. All the examples are made up. 
> #### We look forward to hearing how you apply the examples in this notebook to improve a process within your business! 

Let's Get Started!

In [ ]:
from lamini.classify.lamini_classifier import LaminiClassifier
import random
import textwrap


cls = LaminiClassifier(
    f"CustomerSupportTriageExample{random.randint(1000,9999)}"
)

### Define the classes:
Once the project is created, we define the classes. The more detailed the description, the higher your initial accuracy will be. It helps to give a few examples of keywords or phrases that are likely to appear in this category. You can think of the description as a prompt for Lamini to understand the definition.

A template for how to structure the descriptions follows a typical prompting pattern:  

- A natural language description. 
    - Think about this as how the category might be described to the customer support team in their Standard Operating Procedures (SOPs).  

- Example 1  
    
- Example 2  

In [ ]:

classes = {
    "policy_details": """Chats about policy specifics like coverage, limits, and exclusions. 
    Two examples are: 
    1. What are the details of policy 123456?
    2. Who are the dependents on my current policy?
    """,
    "account_and_billing": """Chat messages asking about how the customer pays, or handling late payments. This could be about policies or other services. 
    For example: 
    1. What is my balance due this month?
    2. I made a payment but I received an email that my balance was past due.
    """,
}


### Add Example(s) for each class

Now that we have a description above for Lamini to use when creating the classifier, we can add additional tagged examples explicitly. These additional examples are training data. Adding example inputs is optional, but will help with accuracy. You can always do this later - we'll add more as a follow up step later in this notebook.

In [ ]:
examples = {
    "policy_details": [
       r'''User: Hi, I'm considering purchasing an auto insurance policy from your company. Could you tell me what types of coverage are included?

Bot: Of course! Our standard auto insurance policy includes liability coverage, collision coverage, comprehensive coverage, uninsured/underinsured motorist coverage, and personal injury protection. Is there a specific type of coverage you would like more details about?

User: I'm particularly interested in understanding the comprehensive coverage. What does it entail?

Bot: Comprehensive coverage helps cover damages to your vehicle that aren't caused by a collision, such as theft, vandalism, fire, or natural disasters. It also covers damage from falling objects or an animal striking your vehicle. Does this align with what you're looking for?

User: yes thanks. Can I set up auto payment for this policy?''',
r'''User: Hi, I'm considering getting a new insurance policy and wanted to understand more about the coverage options available.

Bot: Hello! I'd be happy to help. Our policies typically cover areas such as personal injury, property damage, and liability. Is there a specific type of coverage you're interested in?

User: I'm mainly interested in understanding the coverage for property damage. How extensive is it?

Bot: For property damage, our policies cover a wide range of scenarios, including fire, theft, and natural disasters. You can choose different levels of coverage based on your needs. 

User: That's helpful. How do I determine the right level of coverage for my situation?

Bot: Here is the link.

User: Great! I saw the billing choices are limited and payment is always due the firts monday? Do you have auto pay?''',
r'''User: Hi, I’m considering purchasing home insurance and would like to know more about the coverage options you offer.

Bot: Hello! I'd be happy to help with that. Our home insurance policies can cover a range of incidents including natural disasters, theft, and accidental damage. Is there a specific coverage you are interested in?

User: I’m interested in finding a policy that includes both theft and water damage due to plumbing issues. Is that possible?

Bot: Yes, we do offer policies that include coverage for both theft and water damage caused by plumbing issues. Would you like more detailed information on what each of these covers?

User: No. I want to know how much does it cost? What is the regular payment method?''',
    ],
    "account_and_billing": [
        r'''User: Hi, I have a question about my insurance bill. Could you help me understand the charges for this month?

Bot: Of course! Could you please provide your policy number so I can look up your billing details?

User: Sure, my policy number is 12345678.

Bot: Thank you. I see that this month's charges include your annual premium, which is divided into monthly payments. There's also a small fee for the additional roadside assistance coverage you added last month.

User: Ah, I see now. Thank you for explaining that! Is there a way I can view this breakdown online as well?''',
        r'''User: Hi, I'm trying to understand my recent insurance bill. Can you help me figure it out?

Bot: Of course! I'm here to help. What specific part of the bill are you having trouble with?

User: There's a section labeled ""adjustments"" with an additional charge. I don't understand what that means.

Bot: Adjustments often account for changes in your policy, like a change in coverage or a modification in your payment plan. Would you like me to check if any specific modifications were made to your account?

User: Yes, please. I made a change to my deductible last month, could that be it?''',
        r''' User:  Hi, I have a question about my latest insurance bill. Could you help me figure out why it seems higher than usual?

 Bot:  Of course! I'd be happy to help. Could you provide your policy number so I can take a closer look at your billing details?

 User:  Sure, my policy number is 12345678.

 Bot:  Thank you! After reviewing your account, I see that there was an adjustment made this month due to a recent claim. This might explain the increase. Would you like more details on this adjustment?

 User:  Yes, please. I'd like to understand what the adjustment was for and if my bill will be lower next month.'''
    ],
}


A simple function to print out the examples in a readable format. We will use this again when we add data programmatically a bit later.

Lets review what we have so far.

In [ ]:
# Function to word-wrap and display
def display_wrapped_content(data, width=200):
    wrapper = textwrap.TextWrapper(width=width)
    for key, conversations in data.items():
        print(f"Category: {key}\n" + "="*len(f"Category: {key}"))
        for i, conversation in enumerate(conversations, 1):
            print(f"\nConversation {i}:")
            print(wrapper.fill(conversation))
            print("\n" + "-"*width)
            
#Review the examples from our file
display_wrapped_content(examples)

We have our class descriptions and training examples, time to create the classifier. Use the **.initialize** endpoint to create a new project. This can take about a minute per class, so we'll put in a simple timer to keep us updated on status.

In [ ]:
resp = cls.initialize(classes, examples) 

import time

while True:
    print("Waiting for classifier to initialize")
    time.sleep(5)
    resp = cls.train_status()
    if resp["status"] == "completed":
        print("Model ID: " + resp["model_id"])
        first_model_id = resp["model_id"]
        break
    if resp["status"] == "failed":
        print(resp)
        raise Exception("failed training")


Cool, we have our first model version! Let's try it out with a quick test.

In [134]:
import json

# target: account_and_billing
response = cls.classify(''' 
                        User:  Hi there, I have a question about my insurance bill. Can you help me with that?  \n  \n 
                        Bot:  Absolutely! I’d be happy to assist you with your billing question. What specifically would you like to know?  \n  \n 
                        User:  Could you tell me how I can view and pay my bill online?  \n  \n 
                        Bot:  Sure thing! You can view and pay your bill by logging into your account on our website. Once logged in, navigate to the "Billing" section where you\'ll see options to view your statement and make payments using various methods like credit card or bank transfer.  \n  \n 
                        User:  Great, but I’m not sure I remember my login details. Can you guide me on how to reset my password?
                        ''')


print(json.dumps(response))

{"classification": [[{"class_id": 1, "class_name": "account_and_billing", "prob": 0.651953836198299}, {"class_id": 0, "class_name": "policy_details", "prob": 0.34804616380170095}]]}


Let's take a quick look at the output above. We get a list of all the categories defined in our project, including a confidence score for each.

We can go even further to easily quantify the accuracy of our classifier. Let's run an evaluation!

What an evaluation means for a classifier: when you provide a set of inputs and the expected output, Lamini will test the accuracy of the model on those inputs, and give you back both overall metrics as well as per-input assessment. 

### Build Our Evaluation Dataset

In [ ]:

# account_and_billing eval
account_and_billing_eval = [
    {
        "input": "User: Hi, I'm trying to figure out how to view my recent insurance bill online. Can you help me with that?\n\nBot: Certainly! To view your recent bill, you need to log into your account on our website. Once logged in, navigate to the \"Billing\" section where you can view and download your recent statements.\n\nUser: Okay, but I don't remember my login details. What should I do?\n\nBot: No worries! You can click on the \"Forgot Password\" link on the login page to reset your password. Just follow the instructions sent to your registered email to regain access.\n\nUser: Got it, thanks! Is there any option to set up automatic payments?\n\nBot: Yes, you can set up automatic payments from the \"Payment Settings\" in your account. This allows you to link a bank account or credit card for scheduled payments to ensure your bills are always paid on time.\n\nUser: Great, that sounds convenient. I'll try setting that up. Thanks for your help!",
        "target": "account_and_billing"
    },
    {
        "input": "User: Hi, I have a question about my upcoming insurance bill. When is it due?\n\nBot: Hello! Let me check that for you. Could you please provide your policy number?\n\nUser: Sure, it's 12345678.\n\nBot: Thank you! Your payment is due on the 15th of next month. Would you like to set up a reminder or schedule automatic payments?\n\nUser: No, that's okay. I just need to know if there are any late fees if I miss the due date.",
        "target": "account_and_billing"
    },
    {
        "input": "User: Hi, I need some help understanding my billing statement. Can you help me with that?\n\nBot: Of course! I'd be happy to help. What specifically would you like to know about your billing statement?\n\nUser: I'm not sure about the different charges on my bill. Could you explain what each charge is for?\n\nBot: Sure! Typically, your bill includes your monthly premium, any additional services or riders you have, and any applicable taxes or fees. Is there a particular charge you're curious about?\n\nUser: Yes, there's a charge labeled \"optional coverage fee\" that I'm not familiar with. Can you tell me more about it?",
        "target": "account_and_billing"
    },
    {
        "input": " User:  Hi, I have a question about my insurance bill. I noticed there's been a change in my premium this month. Can you help me understand why?\n\n Bot:  Certainly! Changes in premiums can occur due to a variety of reasons, such as policy updates, changes in coverage, or adjustments based on your risk profile. Would you like me to check your account specifics?\n\n User:  Yes, that would be great. I just want to confirm if it's due to some policy update or something else.\n\n Bot:  I've pulled up your account details, and it looks like the increase is due to a recent policy update focused on enhanced coverage. This update aims to provide more comprehensive benefits in the event of a claim.\n\n User:  Oh, I see. Thanks for clarifying that for me. Is there a way to get notified in advance about these changes?",
        "target": "account_and_billing"
    },
    {
        "input": "User: Hi, can you help me with understanding my insurance bill this month?\n\nBot: Of course! Could you please provide your policy number so I can look up the details?\n\nUser: Sure, my policy number is 123456789.\n\nBot: Thank you! I see your bill includes a premium payment and a small adjustment for a policy change last month. Would you like more details on these charges?\n\nUser: Yes, please explain the adjustment for the policy change.",
        "target": "account_and_billing"
    },
    {
        "input": "User: Hi there! I have a question about my billing statement. \n\nBot: Sure, I'd be happy to help! What specifically would you like to know about your billing statement?\n\nUser: I noticed an extra charge labeled \"service fee\" on my last bill. What is this fee for?\n\nBot: The service fee is typically applied for administrative costs related to managing your policy. However, it shouldn't be unexpected. Would you like me to check if it's a recurring charge on your plan?\n\nUser: Yes, please! I'd appreciate that.",
        "target": "account_and_billing"
    },
    {
        "input": "User: Hi, I'm trying to understand why my premium this month seems higher than usual. Can you help with that?\n\nBot: Of course! There could be a few reasons for an increase in your premium. Have you recently updated your policy or added any new coverage?\n\nUser: Hmm, I don't think I've made any changes. Could it be something else?\n\nBot: It's possible that there were changes in your area that affected your rate or adjustments made at the time of your policy renewal. I can also check if any discounts have expired.\n\nUser: That would be great! Could you take a look at whether any discounts I had have been removed?",
        "target": "account_and_billing"
    }
]

# policy_details eval
policy_details_eval = [
    {
        "input": "User: Hi, I'm interested in knowing more about the coverage details of your home insurance policy. Can you help me with that?\n\nBot: Of course! Our home insurance policy typically covers damages caused by natural disasters, theft, and vandalism. Would you like information on any specific coverage?\n\nUser: That sounds great. Can you also tell me how long the coverage is for once I purchase the policy?\n\nBot: Yes, the standard home insurance policy is usually valid for one year from the date of purchase. You have the option to renew annually. \n\nUser: Perfect, thank you! I'll think about it and possibly reach out for more details.",
        "target": "policy_details"
    },
    {
        "input": "User: Hi, I was wondering if you could tell me about the coverage options available for a comprehensive car insurance policy.\n\nBot: Of course! Our comprehensive car insurance covers a range of incidents, including theft, vandalism, natural disasters, and accidents. It also includes liability coverage for damages to other vehicles or property.\n\nUser: Thanks! How long does a typical policy term last for car insurance?\n\nBot: A standard car insurance policy term is usually 12 months, but we also offer six-month policies if you prefer shorter terms.\n\nUser: That's good to know. Can I make adjustments to the coverage during the policy term, or do I need to wait until renewal?",
        "target": "policy_details"
    },
    {
        "input": "User: Hi, I was wondering if you could give me more details about what kind of coverage your auto insurance policy offers?\n\nBot: Absolutely! Our auto insurance policies typically cover liability, collision, comprehensive, personal injury protection, and uninsured motorist coverage. Is there a specific part of the coverage you're interested in?\n\nUser: I'm particularly interested in the comprehensive coverage. Can you tell me more about what's included in that?\n\nBot: Certainly! Comprehensive coverage helps protect your vehicle against non-collision incidents like theft, vandalism, natural disasters, and falling objects. It’s a great way to have broader protection for your car.\n\nUser: That sounds useful! Could you also let me know how long the policy usually lasts?",
        "target": "policy_details"
    },
    {
        "input": "User: Hi, I was wondering if you could provide me with some details about the coverage offered in your standard homeowner's insurance policy.\n\nBot: Of course! Our standard homeowner's insurance policy typically covers damages to the structure of your home, personal belongings, liability protection, and additional living expenses if your home becomes uninhabitable due to covered damages.\n\nUser: That sounds comprehensive. Can you tell me how long the coverage period usually lasts?\n\nBot: Sure! Our coverage period is typically for one year, but you can renew it annually. We also offer options for multi-year policies if you're interested in extended coverage.\n\nUser: Great, thank you for the information. Is there a way to customize the coverage within that period if I need to make adjustments later on?",
        "target": "policy_details"
    },
    {
        "input": "User: Hi, I'm considering getting a health insurance policy and wanted to know what types of coverage are typically included?\n\nBot: Hello! I'd be happy to help with that. Our standard health insurance policies usually cover doctor visits, hospital stays, prescription medications, and emergency services. Do you need details on additional coverage options?\n\nUser: Yes, that would be great. Are there options for adding dental and vision coverage?\n\nBot: Absolutely, you can add dental and vision coverage to your plan. These typically include regular check-ups, eye exams, and certain corrective procedures. Would you like more information on specific plans for those?\n\nUser: That sounds good. Could you also let me know the typical length of a policy term?",
        "target": "policy_details"
    },
    {
        "input": "User: Hi, I have a question about my insurance payment. Can you help me understand how I can pay my premium online?\n\nBot: Of course! You can pay your premium online through our website or mobile app. Just log in to your account, navigate to the \"Billing\" section, and select \"Make a Payment.\"\n\nUser: Great, but what payment methods do you accept online?\n\nBot: We accept various payment methods online, including credit cards, debit cards, and e-checks. You can choose your preferred option during the payment process.\n\nUser: Is there a way to set up automatic payments so I don't forget next time?\n\nBot: Absolutely! You can set up automatic payments through the same \"Billing\" section. Just opt for the \"Auto-Pay\" feature and follow the steps to link your preferred payment method.\n\nUser: Thanks for the information! I'll set that up right away. Additionally, any new policy to introduce?",
        "target": "policy_details"
    },
    {
        "input": "User: Hi, I'm trying to figure out how to make a payment for my insurance premium. Can you guide me?\n\nBot: Of course! You can make a payment through our online portal, via phone, or by mailing a check. Would you like more details on any of these options?\n\nUser: Yes, I'd like more information about using the online portal.\n\nBot: Sure! To use the online portal, you need to log into your account on our website. Once logged in, navigate to the \"Billing & Payments\" section, and you can pay with a credit card, debit card, or via bank transfer.\n\nUser: Great, I think I can manage that. Meanwhile, any fun detail about the new policy I shall know? ",
        "target": "policy_details"
    }
]

### Run the evaluation

Using the evaluation endpoint, we simply pass the eval dataset we just built to the **eval_data** parameter of the Lamini Evaler.

In [ ]:
from lamini.one_evaler.one_evaler import LaminiOneEvaler

eval = LaminiOneEvaler(
    test_model_id=first_model_id,
    eval_data_id=f"first_eval{random.randint(1000,9999)}",
    eval_data=account_and_billing_eval+policy_details_eval,
    test_eval_type="classifier",
)

full_eval = eval.run()

print(json.dumps(full_eval, indent=2))

Even with just a few examples of each class, the model already got 85% of eval inputs correct. This is captured in the **tuned_accuracy** field of the eval output.

>Best practice: after you've added a few high-quality examples, you should run an eval and carefully review the ground truth labels to make sure they're aligned with the classifier's task and scope - the tags in a general eval set aren't always the best for a narrowly-defined classifier agent to learn from. Don't just review inputs where the assigned class was completely wrong - also review inputs where the classifier's answer is correct but the confidence score is low.

> Reminder: We can view the confidence by using **cls.classify** like we did above.

Lets take a look at which examples the classifier missed. We can use the below function to print out and view the eval examples that the model got wrong.

In [ ]:
def print_missed_evals(eval):
    missed = [
        pred for pred in eval['predictions'] 
        if pred['test_output'] != pred['target']
    ]
    
    if missed:
        print("Missed evals:")
        for prediction in missed:
            print(f"Input: {prediction['input']}")
            print(f"Expected: {prediction['target']}")
            print(f"Predicted: {prediction['test_output']}")
            print("-" * 80)
    else:
        print("No Missed Evals!")     

print_missed_evals(full_eval)

#### What should we do to get higher accuracy?

To Start: More, well curated training data typically helps, lets add a few more training examples. We only added a few for each class so far.

### Load Training Examples from a file:

Most times our data is likely going to be ingested in an unstructured format from our chat application. Let's take a look at how we might stage that data into the right format to be used to train the classifier. 

> Note: This is additional training data, examples we use here should be ideal examples that our Classifier agent is going to learn from.

The below snippet, loads data from a file that is in the data directory of this repo. It is pre-populated with some data for us. As we run the below code, it will print out the loaded examples to review briefly before we train the classifier again (**.initialize** includes the first training run).

In [ ]:
import json
import os

# verify the location of where our examples are stored, there are a few samples here for you already
filepath_with_examples = "./data/insurance_convos.jsonl"

def load_jsonl_file(relpath):
    data = []
    
    #get the current director where the notebook is running
    current_dir = os.path.dirname(os.getcwd())
    
    #add the relative path to the data directory with our filename
    file_path = os.path.join(current_dir, relpath)
    
    #load data and return the examples
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

def write_jsonl_file(filepath, data):
    with open(filepath, 'a') as file:
        for entry in data:
            file.write(json.dumps(entry) + "\n")

def format_examples_from_file(data):
    examples = {
            "policy_details": [
                line['conversation'] for line in data if line['category'] == "policy_details"
            ],
            "account_and_billing": [
                line['conversation'] for line in data if line['category'] == "account_and_billing"
            ],
        }
    return examples

#load data from jsonl file into array of items
loaded_data = load_jsonl_file(filepath_with_examples)

#parse the array into the format required by the Lamini examples endpoint
examples_from_file = format_examples_from_file(loaded_data)

#Review the examples from our file
display_wrapped_content(examples_from_file)

### Review Programmatically Loaded Examples Above

Our new examples look good!

Last time we added training examples, we used examples straight away when creating the classifier. Here we are going to add new examples to the same classifier to provide additional data. We will use the same **cls** object that we created already and call the **.add** method. Passing in a unique name for this dataset and the examples. 

### Train the Classifier with the New Examples

Because we are adding additional data explicitely, we need to call **.train** when we have added all the data we want to add to this next model of our classifier project. 
> Note: We can add more than one dataset before training. Training is fast, typically each iteration can be 1 dataset as we iterate on improving the classifier.

In [ ]:
resp = cls.add(
    f"additional_data{random.randint(1000,9999)}", examples_from_file
)

resp = cls.train()

while True:
    print("Waiting for classifier to train")
    time.sleep(5)
    resp = cls.train_status()
    if resp["status"] == "completed":
        print("Model ID: " + resp["model_id"])
        second_model_id = resp["model_id"]
        break
    if resp["status"] == "failed":
        print(resp["status"])
        raise Exception("failed training")


Great, now we have a second model version in our project! Let's run an eval and compare it to the first version. 

> #### Note: When comparing 2 models, you'll notice we pass a few extra parameters to the Evaler. The test model continues the be the model that we want to understand its behavior (our most recent one). The model we are comparing to, is the base model. 

### Lets see the results from the updated classifier!

In [ ]:
print("Running comparison eval between model versions " + first_model_id + " and " + second_model_id)

eval_2 = LaminiOneEvaler(
    test_model_id=second_model_id,
    eval_data_id=f"second_eval{random.randint(1000,9999)}",
    eval_data=account_and_billing_eval+policy_details_eval,
    test_eval_type="classifier",
    base_model_id=first_model_id,
    sbs=True,
    fuzzy=True,
)

full_eval2 = eval_2.run()

print(json.dumps(full_eval2, indent=2))

### Which eval did the model miss?

The eval output makes it easy to compare model versions overall, and to see exactly where the differences are, so you know exactly where to focus to improve your workflow.

Lets take a look at all of the eval data points above that the classifier did not get correct.

In [ ]:
print_missed_evals(full_eval2)

So our model missed the same two...

### How do we think about getting even higher accuracy!?

1. We should look at the content of the missed input(s), closely
2. Source or generate examples that cover the relevant concepts or structure of the missed input(s)
3. Add them to the classifier as training examples
4. Train
5. Verify with Eval

### Its Your Lucky Day, I took a close look at the above examples for us

> #### If you notice the most recent User response in the chat samples, its subtle at first ... The user changes topics well into the conversation. Sometimes even after the initial conversation has concluded. Ideally, we can have our chat experience identify this and continue the conversation with a relevant response. 

Using the same functions we defined above, lets print out our new dataset that is targeted to help fix this. Aptly named 'same_as_before'_v2 of course. It's also provided in the data directory of this repo.

In [ ]:
# separate file with our new examples
filepath_with_examples = "./data/insurance_convos_v2.jsonl"

#load data from jsonl file into array of items
loaded_data = load_jsonl_file(filepath_with_examples)

#parse the array into the format required by the Lamini examples endpoint
examples_from_V2_file = format_examples_from_file(loaded_data)

#Review the examples from our file
display_wrapped_content(examples_from_V2_file)

### We use the same flow as before, lets add our dataset and then train our model with the updated examples

In [ ]:
resp = cls.add(
    f"additional_data{random.randint(1000,9999)}", examples_from_V2_file
)

resp = cls.train()

while True:
    print("Waiting for classifier to train")
    time.sleep(5)
    resp = cls.train_status()
    if resp["status"] == "completed":
        print("Model ID: " + resp["model_id"])
        third_model_id = resp["model_id"]
        break
    if resp["status"] == "failed":
        print(resp["status"])
        raise Exception("failed training")

### As you guessed, now we run eval to see the models performance side by side

This time between model version 2 and the current version 3.

In [ ]:
print("Running comparison eval between model versions " + second_model_id + " and " + third_model_id)

eval_3 = LaminiOneEvaler(
    test_model_id=third_model_id,
    eval_data_id=f"second_eval{random.randint(1000,9999)}",
    eval_data=account_and_billing_eval+policy_details_eval,
    test_eval_type="classifier",
    base_model_id=second_model_id,
    sbs=True,
    fuzzy=True,
)

full_eval3 = eval_3.run()

print(json.dumps(full_eval3, indent=2))

In [ ]:
print_missed_evals(full_eval3)

## Excellent! 
### Our targeted new examples were able to help the model categorize a tricky change in topic correctly now! Great Job!

> ### **Challenge**: Can you add a few more examples to get the model to 100%. Share your training data and results with us to receive an extra $100 in credits on app.lamini.ai

# Happy Classifying!